# Data preparation

In [1]:
import pandas as pd
import plotly.graph_objects as go

## Import data

In [2]:
# Create empty DataFrame to be filled with data
df_raw = pd.DataFrame(columns=["timestamp", "customer_no", "location"])

# Import data
days = ["monday", "tuesday", "wednesday", "thursday", "friday"]

# Loop through days and import one csv file per day
for i, day in enumerate(days):
    data = pd.read_csv(f"../data/{day}.csv", delimiter=";", parse_dates=[0])

    # Add 10,000 per day to customer number to separate days
    data["customer_no"] = data["customer_no"] + (i + 1) * 10000

    df_raw = pd.concat([df_raw, data])

In [3]:
# Sort, reset index and save a copy
df_raw = df_raw.sort_values("timestamp").reset_index(drop=True)
df = df_raw.copy()

## For every customer, add a line for entrance

In [4]:
# Create a copy of a grouped dataframe (by customer_no) and get first values
first_entry = df.groupby("customer_no").first().reset_index().copy()

# Change timestamp minus one minute
first_entry["timestamp"] = first_entry["timestamp"] - pd.Timedelta(minutes=1)

# Set location to entrance
first_entry["location"] = "entrance"

# Add dataframe as rows to the old dataframe
df = pd.concat([df, first_entry])

# Sort by timestamp and reset index
df = df.sort_values("timestamp").reset_index(drop=True)

## Find customers with mising checkout

In [5]:
# Create a copy of a grouped dataframe (by customer_no) and get last values
last_locations = df.groupby("customer_no").last().reset_index().copy()

# Filter out those customers that have no checkout as last location
last_locations = last_locations[last_locations["location"] != "checkout"]

# Add one minute to timestamp
last_locations["timestamp"] = last_locations["timestamp"] + pd.Timedelta(minutes=1)

# Set location to checkout
last_locations["location"] = "checkout"

# Add dataframe as rows to the old dataframe
df = pd.concat([df, last_locations])

# Sort by timestamp and reset index
df = df.sort_values("timestamp").reset_index(drop=True)

## Fill missing datetimes

In [6]:
# See entries for one customer before
df[df["customer_no"] == 10434]

,timestamp,customer_no,location
1875,2019-09-02 11:54:00,10434,entrance
1878,2019-09-02 11:55:00,10434,fruit
1883,2019-09-02 11:56:00,10434,dairy
1887,2019-09-02 11:57:00,10434,fruit
1892,2019-09-02 11:58:00,10434,dairy
1898,2019-09-02 11:59:00,10434,drinks
1918,2019-09-02 12:04:00,10434,fruit
1937,2019-09-02 12:08:00,10434,checkout


In [7]:
df = (
    df.set_index("timestamp")
    .groupby("customer_no")[["location"]]
    .resample(rule="1min")
    .ffill()
    .reset_index()
)

In [8]:
# See entries for one customer after
df[df["customer_no"] == 10434]

,customer_no,timestamp,location
3682,10434,2019-09-02 11:54:00,entrance
3683,10434,2019-09-02 11:55:00,fruit
3684,10434,2019-09-02 11:56:00,dairy
3685,10434,2019-09-02 11:57:00,fruit
3686,10434,2019-09-02 11:58:00,dairy
3687,10434,2019-09-02 11:59:00,drinks
3688,10434,2019-09-02 12:00:00,drinks
3689,10434,2019-09-02 12:01:00,drinks
3690,10434,2019-09-02 12:02:00,drinks
3691,10434,2019-09-02 12:03:00,drinks


In [9]:
# Save the cleaned data to file
df.to_csv("../data/data_clean.csv")

## Calculate transition probabilities 

In [10]:
# Add shifted columns
df["loc_next"] = df.groupby("customer_no")["location"].shift(-1)
# df["loc_last"] = df.groupby("customer_no")["location"].shift(1)
# df["loc_next"].fillna("checkout", inplace=True)
# df["loc_last"].fillna("entrance", inplace=True)

In [11]:
P = pd.crosstab(
    index=df["location"],
    columns=df["loc_next"],
    normalize="index",
)

# Add checkout row
checkout = pd.DataFrame(
    {"checkout": [1], "dairy": [0], "drinks": [0], "fruit": [0], "spices": [0]},
    index=["checkout"],
)
P = pd.concat([P, checkout])

# Add entrance column
P["entrance"] = 0.0

P = P[["entrance", "dairy", "drinks", "fruit", "spices", "checkout"]]

In [12]:
P

,entrance,dairy,drinks,fruit,spices,checkout
dairy,0.0,0.736750,0.058569,0.049848,0.051367,0.103466
drinks,0.0,0.010899,0.598499,0.087909,0.086983,0.215710
entrance,0.0,0.287576,0.153526,0.377435,0.181464,0.000000
fruit,0.0,0.095924,0.054847,0.596947,0.050677,0.201605
spices,0.0,0.193214,0.163109,0.090953,0.402039,0.150685
checkout,0.0,0.000000,0.000000,0.000000,0.000000,1.000000


In [13]:
# Save transition probabilities to csv
P.to_csv("../data/transition_probabilities.csv")

### Also export not normalized data

In [14]:
P2 = pd.crosstab(
    index=df["location"],
    columns=df["loc_next"],
)

# Save the transition probabilities to a csv file
P2.to_csv("../data/transition_probabilities_count.csv")

In [15]:
P2

loc_next,checkout,dairy,drinks,fruit,spices
location,,,,,
dairy,1839,13095,1041,886,913
drinks,2098,106,5821,855,846
entrance,0,2141,1143,2810,1351
fruit,2562,1219,697,7586,644
spices,946,1213,1024,571,2524


## Visualize missing dates

In [16]:
# Filter dataframes to only include part of the customers
df_plot_before = df_raw[df_raw["customer_no"] < 10100].copy()
df_plot_after = df[df["customer_no"] < 10100].copy()

# Map location to integer
df_plot_before["z"] = df_plot_before["location"].map(
    {"entrance": 1, "dairy": 2, "drinks": 3, "fruit": 4, "spices": 5, "checkout": 6}
)
df_plot_after["z"] = df_plot_after["location"].map(
    {"entrance": 1, "dairy": 2, "drinks": 3, "fruit": 4, "spices": 5, "checkout": 6}
)

In [17]:
# Plot the data before missing values have been imputed
fig = go.Figure(
    go.Heatmap(
        y=df_plot_before["customer_no"],
        x=df_plot_before["timestamp"],
        z=df_plot_before["z"],
    )
)

fig.update_layout(
    title="Before missing values are filled (selection)",
    height=600,
    width=1000,
    yaxis_autorange="reversed",
)

fig.show()

In [18]:
# Plot the data after missing values have been imputed
fig = go.Figure(
    go.Heatmap(
        y=df_plot_after["customer_no"],
        x=df_plot_after["timestamp"],
        z=df_plot_after["z"],
    )
)

fig.update_layout(
    title="After missing values have been imputed (selection)",
    height=600,
    width=1000,
    yaxis_autorange="reversed",
)

fig.show()